In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os
# os.chdir('/content/drive/Shareddrives/hoa.lenghiem/ThanhQuang_NLP')
os.chdir('/content/drive/MyDrive/VIN_NLP/ThanhQuang_NLP')

In [3]:
!pip install pyvi
!pip install https://gitlab.com/trungtv/vi_spacy/-/raw/master/vi_core_news_lg/dist/vi_core_news_lg-0.0.1.tar.gz

     |████████████████████████████████| 8.5 MB 14.5 MB/s 
     |████████████████████████████████| 743 kB 68.6 MB/s 
     |████████████████████████████████| 254.5 MB 39 kB/s 
     |████████████████████████████████| 6.3 MB 8.9 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 10.1 MB 65.1 MB/s 
     |████████████████████████████████| 451 kB 64.4 MB/s 
     |████████████████████████████████| 628 kB 61.2 MB/s 
  Created wheel for vi-core-news-lg: filename=vi_core_news_lg-0.0.1-py3-none-any.whl size=254513617 sha256=aedd67a10ba76bd91cad8cbd361749e39a75129c5466860594740e654792c87b
  Stored in directory: /root/.cache/pip/wheels/e4/d9/90/dcbb25186a2c3335b17bc675f8dddcb43ecdfa4b400e8c91b0
Successfully built vi-core-news-lg
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installatio

In [4]:
# required libraries

import unicodedata
import string
import re
import random
import time
import datetime
import math
import pickle

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import torchtext
from torchtext.legacy.datasets import TranslationDataset

import spacy
import numpy as np
from nltk.translate.bleu_score import corpus_bleu

# set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
from pyvi import ViTokenizer

'''
tokenization code
'''

import spacy
spacy_vi = spacy.load('vi_core_news_lg')
stopwords = spacy_vi.Defaults.stop_words

def tokenize_vi(text):
    """
    Tokenizes Vietnamese text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_vi.tokenizer(text)]

# #Xử lý các từ lặp
# def remove(txt):
#     txt = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), str(txt), flags=re.IGNORECASE)
#     return txt

# def A_cvt_a(txt):
#     return txt.lower()

# # def utils_data(txt):
# #     pat = re.compile(r"\b(%s)\b" % "|".join(acronyms))
# #     txt = pat.sub(lambda m: acronyms.get(m.group()), txt)
# #     return txt

# #Xử lý các từ dừng
# def remove_stopwords1(txt):
#     txt = ViTokenizer.tokenize(txt)
#     split_words = []
#     words = []
#     try:
#         split_words = [x.strip('0123456789%@$.,=+-!;/()*"&^:#|\n\t\'').lower() for x in txt.split()]
#     except:
#         pass
#     for w in split_words:
#         if w not in stopwords:
#             words.append(w)
#     return " ".join(words)

# #Xử lý các từ dừng
# def remove_stopwords(txt):
#     split_words = txt.split()
#     final_txt = []
#     for ch in split_words:
#         if ch not in stopwords:
#             final_txt.append(ch)
#     return " ".join(final_txt)

# #Xử lý dấu câu
# def remove_punctuations(txt):
#     punctuations = '@#!?+&*[]-%:/();$=><|{}^_' + "'`"
#     for p in punctuations:
#         txt = txt.replace(p, f' {p} ')
#     return txt

# def preProcess(txt):
#     txt = remove(txt)
#     txt = A_cvt_a(txt)
#     # txt = utils_data(txt)
#     #Xóa các kí tự xuống dòng
#     txt = " ".join(re.sub("\n", " ", txt).split())
#     #Xử lý dấu câu
#     txt = remove_punctuations(txt)
#     # #Tách từ
#     txt = " ".join(tokenize_vi(txt))
#     # #Xóa từ dừng
#     # txt = remove_stopwords(txt)
#     # txt = remove_stopwords1(txt)
#     return txt

In [ ]:
# !wget https://public.vinai.io/word2vec_vi_words_300dims.zip
# !unzip -q word2vec_vi_words_300dims

In [ ]:
with open("word2vec_vi_words_300dims.txt", mode="r", encoding="utf8") as fi:
    # Chỉ đọc dòng đầu tiên
    first_line = fi.readline()
    total_tokens = first_line.split(" ")[0]
    dims = first_line.split(" ")[1]
    print(f"There are {total_tokens} words/tokens in the corpus, each word/token is represented by a {dims}-dimensional vector")

There are 1587507 words/tokens in the corpus, each word/token is represented by a 300
-dimensional vector


In [9]:
'''
define field
'''
SRC = torchtext.legacy.data.Field(
    # tokenize = tokenize_vi, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)
TRG = torchtext.legacy.data.Field(
    # tokenize = tokenize_vi, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [12]:
'''
load the data
'''
from torchtext.legacy import data
from torchtext.legacy import datasets

train_data = torchtext.legacy.data.TabularDataset(
    path='final_paws_translated_data/train_preprocessed.csv',
    format='csv', skip_header=True, fields=[('preprocessed_sent1', SRC), 
                                            ('preprocessed_sent2', TRG), 
                                            # ('label', LABEL)
                                            ],
                                            )

print(f"Number of training examples: {len(train_data.examples)}")

'''
build the vocabulary
'''
vectors = torchtext.vocab.Vectors("word2vec_vi_words_300dims.txt")
TRG.build_vocab(train_data, vectors=vectors)
SRC.build_vocab(train_data, vectors=vectors)
print(f"Unique tokens in source (vi) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (vi) vocabulary: {len(TRG.vocab)}")

'''
create the iterator
'''
train_iter = torchtext.legacy.data.BucketIterator(train_data, batch_size=64, device=device, 
                                                  sort_key=lambda x: len(x.preprocessed_sent1), 
                                                  sort_within_batch=True,
                                                  shuffle=True)

Number of training examples: 25670
Unique tokens in source (vi) vocabulary: 24478
Unique tokens in target (vi) vocabulary: 24448


In [ ]:
type(train_iter)

torchtext.legacy.data.iterator.BucketIterator

In [ ]:
print(vars(train_data[0]))

{'preprocessed_sent1': ['ngoài_ra', 'còn', 'có', 'các', 'cuộc', 'thảo_luận', 'cụ_thể', 'các', 'cuộc', 'tranh_luận', 'hồ_sơ', 'công_khai', 'và', 'các', 'cuộc', 'thảo_luận', 'dự_án', '.'], 'preprocessed_sent2': ['ngoài_ra', 'còn', 'có', 'các', 'cuộc', 'thảo_luận', 'công_khai', 'các', 'cuộc', 'thảo_luận', 'cụ_thể', 'hồ_sơ', 'và', 'các', 'cuộc', 'thảo_luận', 'dự_án', '.']}


In [ ]:
vectors.get_vecs_by_tokens(vars(train_data[0])['preprocessed_sent1'])

tensor([[ 0.0346, -0.0311, -0.0109,  ..., -0.0203,  0.1279, -0.0144],
        [ 0.0082, -0.0306,  0.0511,  ..., -0.0207,  0.0302,  0.0298],
        [ 0.0141,  0.0126,  0.0563,  ...,  0.0028,  0.0256,  0.0230],
        ...,
        [-0.0041, -0.0083,  0.0215,  ..., -0.0303, -0.0162,  0.0241],
        [ 0.0111, -0.0339, -0.0030,  ..., -0.0541,  0.0298, -0.0454],
        [-0.0332, -0.0188,  0.0274,  ..., -0.0555, -0.0464, -0.0233]])

In [ ]:
vectors.get_vecs_by_tokens(["."])

tensor([[-3.3222e-02, -1.8811e-02,  2.7442e-02,  2.3059e-02,  3.6801e-02,
         -2.6694e-02, -2.4474e-02, -6.8289e-02,  4.8087e-02, -1.3581e-01,
         -3.2308e-02, -3.3542e-02, -2.2968e-03,  4.5109e-02,  6.3254e-02,
         -1.0021e-01,  9.0904e-03,  1.3119e-02, -4.5714e-02,  4.0472e-02,
         -3.9495e-02, -4.3859e-02,  6.8815e-02, -5.8470e-02, -1.4110e-02,
         -6.4529e-02, -1.3494e-01,  1.4276e-02,  2.2320e-02, -1.5523e-02,
         -3.4654e-04,  2.3541e-02,  6.8071e-02,  3.9035e-02,  3.2343e-02,
         -8.8038e-02,  7.3987e-02,  8.2950e-02,  8.9225e-02, -5.1089e-02,
          4.0799e-06,  1.6441e-02, -6.3483e-02,  3.5130e-02,  4.4692e-02,
         -1.0251e-02, -3.6713e-03, -2.9649e-02,  1.6578e-01, -5.8407e-03,
          3.8981e-02,  1.1178e-01,  3.4866e-02, -8.2145e-04,  3.3404e-02,
         -3.1747e-02,  3.1505e-02, -1.6732e-02,  2.6087e-02, -1.8300e-02,
         -2.3825e-02, -4.1885e-03,  1.2503e-02, -4.2940e-02, -5.0484e-02,
         -3.6145e-02,  1.6749e-01,  3.

# Word embedding using pre-trained model PhoW2V

In [ ]:
# save the field
with open("SRC.Field","wb")as f:
     pickle.dump(SRC,f)
with open("TRG.Field","wb")as f:
     pickle.dump(TRG,f)

In [7]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim,n_layers, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dropout = dropout
        self.n_layers = n_layers

        # self.embedding = nn.Embedding(input_dim, emb_dim)
        # further you can now make use of your pretrained embeddings inside your model by passing them in
        # as a parameter or loading the handy pickle file mentioned above - a much better design pattern
        # indeed than relying on TEXT.build_vocab() in order to define the embedding layer of your model
        self.embedding = nn.Embedding.from_pretrained(SRC.vocab.vectors, freeze=True)
        # freeze=True <-> self.embedding.requires_grad = False, not learn in training
        # SRC.vocab.vectors, freeze=False

        self.lstm = nn.LSTM(emb_dim, enc_hid_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.lstm(embedded)
        
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, dec_hid_dim, n_layers, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.output_dim = output_dim
        self.dec_hid_dim = dec_hid_dim
        self.n_layers = n_layers
        self.dropout = dropout

        # self.embedding = nn.Embedding(output_dim, emb_dim)
        self.embedding = nn.Embedding.from_pretrained(TRG.vocab.vectors, freeze=True)
        # self.embedding.requires_grad = False
        self.lstm = nn.LSTM(emb_dim, dec_hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(dec_hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        input = input.unsqueeze(0)
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]    
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        
        # predicted shape is [batch_size, output_dim]
        prediction = self.fc_out(hidden.squeeze(0))
        
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    ''' 
    Args:
        encoder: A Encoder class instance.
        decoder: A Decoder class instance.
    '''
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        # to store the outputs of the decoder
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)

        # first input to the decoder is the <sos> tokens
        input = trg[0, :]

        for t in range(1, max_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            use_teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input = (trg[t] if use_teacher_force else top1)

        # outputs is of shape [sequence_len, batch_size, output_dim]
        return outputs

In [ ]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
print(TRG_PAD_IDX)

1


In [ ]:
import os
'''
hyperparameters
'''
INPUT_DIM = len(SRC.vocab)
print(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 300
DEC_EMB_DIM = 300
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
N_LAYERS = 1
LEARNING_RT = 0.001
N_EPOCHS = 50
CLIP = 1

'''
instantiate the model
'''
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT)
model = Seq2Seq(enc, dec,device).to(device)
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RT)
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]
print('<pad> token index: ',TRG_PAD_IDX)
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

'''
initialize the model weights
'''
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
model.apply(init_weights)

'''
calculate the number of parameters
'''
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters')

'''
compute the loss function with first training batch
'''
clip = 1
model.train()



for i, batch in enumerate(train_iter):

    src = batch.preprocessed_sent1
    trg = batch.preprocessed_sent2
    optimizer.zero_grad()

    output = model(src, trg)

    output_dim = output.shape[-1]

    output = output[1:].view(-1, output_dim)
    trg = trg[1:].view(-1)

    loss = criterion(output, trg)

    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)

    optimizer.step()
    
    print('loss from first training batch')
    print(loss/src.shape[1])
    break

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


<pad> token index:  1
The model has 15,875,968 trainable parameters
loss from first training batch
tensor(0.1579, device='cuda:0', grad_fn=<DivBackward0>)


In [ ]:
'''
Full training helper functions
'''

def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.preprocessed_sent1
        trg = batch.preprocessed_sent2
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
      
        torch.nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.preprocessed_sent1
            trg = batch.preprocessed_sent2

            output = model(src, trg, 0) #turn off teacher forcing

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

# convert index to text string
def convert_itos(convert_vocab, token_ids):
    list_string = []
    for i in token_ids:
        if i == convert_vocab.vocab.stoi['<eos>']:
            break
        else:
            token = convert_vocab.vocab.itos[i]
            list_string.append(token)
    return list_string
    

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
'''
kickstart full training
'''

best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)

    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # Create checkpoint at end of each epoch
    state_dict_model = model.state_dict() 
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
        }

    if not os.path.isdir("ckpt_seq2seq"):
        os.mkdir("ckpt_seq2seq")
    torch.save(state, "ckpt_seq2seq/seq2seq_"+str(epoch+1)+".pt")

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')


Epoch: 01 | Time: 2m 20s
	Train Loss: 7.052 | Train PPL: 1154.818
Epoch: 02 | Time: 2m 20s
	Train Loss: 6.724 | Train PPL: 832.534
Epoch: 03 | Time: 2m 19s
	Train Loss: 6.619 | Train PPL: 749.464
Epoch: 04 | Time: 2m 20s
	Train Loss: 6.552 | Train PPL: 700.485
Epoch: 05 | Time: 2m 19s
	Train Loss: 6.393 | Train PPL: 597.538


# Resuming

In [ ]:
'''
load fields saved during preprocessing
'''
# save the field
with open("SRC.Field","wb")as f:
     pickle.dump(SRC,f)
with open("TRG.Field","wb")as f:
     pickle.dump(TRG,f)
'''
hyperparameters (ensure the following hyperparameters match with those used during training of the best model)
'''
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 300
DEC_EMB_DIM = 300
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
N_LAYERS = 1

'''
instantiate the model
'''
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT)
model_best = Seq2Seq(enc, dec,device).to(device)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
'''
load the checkpoint corresponding to the best epoch (usually epoch with highest validation BLEU score)
'''
ckpt_dir_path = 'ckpt_seq2seq'
ckpt_name = 'seq2seq_50.pt'
model_best.load_state_dict(torch.load(os.path.join(ckpt_dir_path, ckpt_name))['state_dict'])
model = model_best.to(device)
curr_epoch = ckpt_name.split("_")[1].split(".")[0]
print("Current saved epoch: ", curr_epoch)
curr_epoch = int(curr_epoch)

Current saved epoch:  50


In [ ]:
'''
kickstart full training
'''

best_valid_loss = float('inf')
N_EPOCHS = 50
LEARNING_RT = 0.001
CLIP = 1
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RT)
for epoch in range(curr_epoch, 4*N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)

    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # Create checkpoint at end of each epoch
    state_dict_model = model.state_dict() 
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
        }

    if not os.path.isdir("ckpt_seq2seq"):
        os.mkdir("ckpt_seq2seq")
    torch.save(state, "ckpt_seq2seq/seq2seq_"+str(epoch+1)+".pt")

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')

Epoch: 51 | Time: 1m 3s
	Train Loss: 0.529 | Train PPL:   1.697
Epoch: 52 | Time: 1m 3s
	Train Loss: 0.483 | Train PPL:   1.621
Epoch: 53 | Time: 1m 3s
	Train Loss: 0.471 | Train PPL:   1.601
Epoch: 54 | Time: 1m 3s
	Train Loss: 0.468 | Train PPL:   1.597
Epoch: 55 | Time: 1m 3s
	Train Loss: 0.463 | Train PPL:   1.589
Epoch: 56 | Time: 1m 3s
	Train Loss: 0.452 | Train PPL:   1.572
Epoch: 57 | Time: 1m 3s
	Train Loss: 0.456 | Train PPL:   1.578
Epoch: 58 | Time: 1m 3s
	Train Loss: 0.444 | Train PPL:   1.559
Epoch: 59 | Time: 1m 3s
	Train Loss: 0.429 | Train PPL:   1.536
Epoch: 60 | Time: 1m 3s
	Train Loss: 0.440 | Train PPL:   1.552
Epoch: 61 | Time: 1m 3s
	Train Loss: 0.431 | Train PPL:   1.539
Epoch: 62 | Time: 1m 3s
	Train Loss: 0.423 | Train PPL:   1.527
Epoch: 63 | Time: 1m 3s
	Train Loss: 0.409 | Train PPL:   1.505
Epoch: 64 | Time: 1m 3s
	Train Loss: 0.408 | Train PPL:   1.504
Epoch: 65 | Time: 1m 3s
	Train Loss: 0.406 | Train PPL:   1.501
Epoch: 66 | Time: 1m 3s
	Train Loss: 0.3

In [ ]:
'''
load the checkpoint corresponding to the best epoch (usually epoch with highest validation BLEU score)
'''
ckpt_dir_path = 'ckpt_seq2seq'
ckpt_name = 'seq2seq_200.pt'
model.load_state_dict(torch.load(os.path.join(ckpt_dir_path, ckpt_name))['state_dict'])
model = model.to(device)
curr_epoch = ckpt_name.split("_")[1].split(".")[0]
print("Current saved epoch: ", curr_epoch)

Current saved epoch:  200


In [ ]:
'''
kickstart full training
'''

best_valid_loss = float('inf')

curr_epoch = int(curr_epoch)

for epoch in range(curr_epoch, curr_epoch+N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)

    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    # Create checkpoint at end of each epoch
    state_dict_model = model.state_dict() 
    state = {
        'epoch': epoch,
        'state_dict': state_dict_model,
        'optimizer': optimizer.state_dict()
        }

    if not os.path.isdir("ckpt_seq2seq"):
        os.mkdir("ckpt_seq2seq")
    torch.save(state, "ckpt_seq2seq/seq2seq_"+str(epoch+1)+".pt")

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')

Epoch: 201 | Time: 1m 6s
	Train Loss: 0.305 | Train PPL:   1.356
Epoch: 202 | Time: 1m 7s
	Train Loss: 0.315 | Train PPL:   1.370
Epoch: 203 | Time: 1m 7s
	Train Loss: 0.303 | Train PPL:   1.354
Epoch: 204 | Time: 1m 6s
	Train Loss: 0.297 | Train PPL:   1.346
Epoch: 205 | Time: 1m 6s
	Train Loss: 0.292 | Train PPL:   1.340
Epoch: 206 | Time: 1m 6s
	Train Loss: 0.297 | Train PPL:   1.345
Epoch: 207 | Time: 1m 7s
	Train Loss: 0.294 | Train PPL:   1.342
Epoch: 208 | Time: 1m 7s
	Train Loss: 0.305 | Train PPL:   1.356
Epoch: 209 | Time: 1m 6s
	Train Loss: 0.292 | Train PPL:   1.339
Epoch: 210 | Time: 1m 6s
	Train Loss: 0.288 | Train PPL:   1.333
Epoch: 211 | Time: 1m 6s
	Train Loss: 0.290 | Train PPL:   1.336
Epoch: 212 | Time: 1m 6s
	Train Loss: 0.294 | Train PPL:   1.342
Epoch: 213 | Time: 1m 7s
	Train Loss: 0.300 | Train PPL:   1.349
Epoch: 214 | Time: 1m 7s
	Train Loss: 0.300 | Train PPL:   1.349
Epoch: 215 | Time: 1m 6s
	Train Loss: 0.292 | Train PPL:   1.339
Epoch: 216 | Time: 1m 7s


# Predictions

In [13]:
'''
load fields saved during preprocessing
'''
with open("TRG.Field","rb") as f:
     TRG_saved = pickle.load(f)

with open("SRC.Field","rb") as f:
     SRC_saved = pickle.load(f)

'''
hyperparameters (ensure the following hyperparameters match with those used during training of the best model)
'''
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 300
DEC_EMB_DIM = 300
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
N_LAYERS = 1

'''
instantiate the model
'''
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT)
model_best = Seq2Seq(enc, dec, device).to(device)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [14]:
'''
load the checkpoint corresponding to the best epoch (usually epoch with highest validation BLEU score)
'''
# n_ckpt = 200
n_ckpt = 200
model_best.load_state_dict(torch.load(f'ckpt_seq2seq/seq2seq_{n_ckpt}.pt')['state_dict'])
model_best = model_best.to(device)

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
os.chdir('/content/drive/MyDrive/VIN_NLP/ThanhQuang_NLP')

In [17]:
test_data = torchtext.legacy.data.TabularDataset(
    path="test_paraphrase.csv",
    format='csv', skip_header=True, fields=[('src', SRC)]) 

In [18]:
test_iter = torchtext.legacy.data.Iterator(test_data, batch_size=256, device=device, sort=False, sort_key=None, shuffle=False, sort_within_batch=False)

In [19]:
# convert index to text string
def convert_itos(convert_vocab, token_ids):
    list_string = []
    for i in token_ids:
        if i == convert_vocab.vocab.stoi['<eos>']:
            break
        else:
            token = convert_vocab.vocab.itos[i]
            list_string.append(token)
    return list_string

In [25]:
'''
generate paraphrases for all the sentences in test data
'''
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

def generate_paraphrases(model, eval_iter, trg_vocab, attention = True, max_trg_len = 64):
    model.eval()
    all_translation_word_ids = []
    for batch in test_iter:
        src = batch.src
        batch_size = src.shape[1]


        trg_placeholder = torch.Tensor(max_trg_len, batch_size)
        trg_placeholder.fill_(TRG_PAD_IDX)
        trg_placeholder = trg_placeholder.long().to(device)
        if attention == True:
          output,_ = model(src, trg_placeholder, 0) #turn off teacher forcing
        else:
          output = model(src, trg_placeholder, 0) #turn off teacher forcing
        
        output_translate = output[1:]

        # Choose top 1 word from decoder's output, we get the probability and index of the word
        prob, token_id = output_translate.data.topk(1)
        translation_token_id = token_id.squeeze(2).cpu()

        # store gold target sentences to a list 
        all_translation_word_ids.append(translation_token_id)

    all_translation_text = []
    for i in range(len(all_translation_word_ids)):
      cur_translation_batch = all_translation_word_ids[i]
      for j in range(cur_translation_batch.shape[1]):
        trans_convered_strings = convert_itos(trg_vocab, cur_translation_batch[:,j])
        all_translation_text.append(' '.join(trans_convered_strings)) # convert list of words to text
    
    return all_translation_text

In [26]:
# translate all the sentences in the test set    
test_predictions = generate_paraphrases(model_best, test_iter, TRG_saved, attention = False, max_trg_len = 64)

In [27]:
import pandas as pd
test_csv = pd.read_csv("test_paraphrase.csv")["src"].tolist()
test_csv

['Để bán được hàng thàng công , trước tiên bạn phải yêu quý công việc và hết lòng vì nó . ',
 'Điều này cho phép bạn lựa chọn những công việc cần phải hoàn thành trong ngày hôm đó và loại bỏ các công việc có thể hoàn thành vào các ngày khác .',
 'Vì vậy , bạn hãy học cách nói “ không ” với những yêu cầu có mức ưu tiên thấp , và bạn sẽ có nhiều thời gian để làm những việc quan trọng hơn .',
 'Để có được những thành tựu như ngày hôm nay , ngoài những giải pháp chung đã được nhiều tỉnh thành trên cả nước áp dụng , Đà Nẵng còn có những “ chiêu ” chưa hề có tiền lệ , góp phần phát triển kinh tế một cách mạnh mẽ và tạo ra sự ổn định trong đời sống xã hội .',
 'Không dừng lại ở bạn bè , thầy cô , mà ngay cả người thân trong gia đình cũng bị không ít bạn trẻ dùng những từ ngữ “ vô học ” công kích trên mạng . ',
 'Chúng ta có nhiều giống cây , con bản địa quý , mang hương vị Việt Nam , nhưng đã bị lãng quên nhiều năm , và nhiều giống đã vĩnh viễn biến mất . ',
 'Còn nhà mạng , vốn cũng là một d

In [30]:
for index,(orig_sent, pred_sent) in enumerate(zip(test_csv,test_predictions)):
  if index == 7:
    print("orig: ", orig_sent)
    print("pred: ", pred_sent)

orig:  Nghiên cứu phụ nữ Việt Nam là lãnh vực chưa có nhiều người đi sâu .
pred:  chúng bắt_nguồn từ tư_cách là một đô_thị với lá cao nên được gửi đến người .


In [ ]:
test_predictions

['1995 khôi_phục bộ_luật juliána anicia dioscurides tái_biên_soạn bảng nội_dung kinh_thánh nổi_tiếng phát_triển hy lạp la mã nổi_tiếng dòng sr .',
 '1406 khôi_phục bộ_luật juliána anicia dioscurides tái_biên_soạn dập tắt cũ cũ hy lạp la mã trung_thành nghiên_cứu hướng la mã hy lạp cotages rhubarb shed cafe in .',
 '1995 khôi_phục bộ_luật juliána anicia dioscurides tái_biên_soạn cung_cấp nội_dung trang_phục kỹ_thuật cũ zurich phòng phần_mềm phòng nghiên_cứu phòng ngôn_ngữ quốc_gia nga nghiên_cứu châu phi nghiên_cứu ven biển dọc khắp thế_giới ven ven biển .',
 'ethel page israel trang_web corporation phó_chủ_tịch phó_chủ_tịch joseph sở cũ cũ lịch_sử mỹ mùa đông cộng_đồng cộng_đồng ben ben điều_hành cộng_đồng the the litle jewish the jewish the the fre of the crek fre of the litle fre state of the 19 .',
 'jovan quyết_định scherer trao quản_lý truyền_thống home david khôi_phục trang_phục tư_nhân cũ tương_tác đế_quốc hài nghiên_cứu quỹ kỹ_thuật lựa_chọn kỹ_thuật phòng di_sản quân_sự the th